In [34]:
!pip install -U langchain-google-genai langchain_community langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.1 MB/s eta 0:00:00


In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

Enter API key for Google Gemini: ··········


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

genai_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

genai_llm

ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.5, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7dabdfd56480>, default_metadata=(), model_kwargs={})

In [4]:
response = genai_llm.invoke('How far is moon from the earth?')
response

AIMessage(content="The average distance from the Earth to the Moon is about **384,400 kilometers (238,900 miles)**.\n\nIt's important to note that this is an **average**. The Moon's orbit around the Earth is not a perfect circle, but rather an ellipse. This means the distance varies:\n\n*   **Perigee:** The point in its orbit when the Moon is closest to Earth, about 363,300 km (225,700 miles).\n*   **Apogee:** The point in its orbit when the Moon is farthest from Earth, about 405,500 km (251,900 miles).", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--2313e91b-1905-4253-9f2b-76b773dc1843-0', usage_metadata={'input_tokens': 9, 'output_tokens': 151, 'total_tokens': 160, 'input_token_details': {'cache_read': 0}})

In [5]:
print(response.content)

The average distance from the Earth to the Moon is about **384,400 kilometers (238,900 miles)**.

It's important to note that this is an **average**. The Moon's orbit around the Earth is not a perfect circle, but rather an ellipse. This means the distance varies:

*   **Perigee:** The point in its orbit when the Moon is closest to Earth, about 363,300 km (225,700 miles).
*   **Apogee:** The point in its orbit when the Moon is farthest from Earth, about 405,500 km (251,900 miles).


In [17]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage(content="You are a helpful assistant!"),
    HumanMessage(content="What is your name?"),
]

response = genai_llm.invoke(messages)
response

AIMessage(content='I do not have a name.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--30275573-4186-4c22-adb3-85241751576e-0', usage_metadata={'input_tokens': 12, 'output_tokens': 7, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}})

In [18]:
print(response.content)

I do not have a name.


In [31]:
from pydantic import BaseModel, Field
class ResponseFormatter(BaseModel):
    name: str = Field(description="Name of the user")
    gender: str = Field(description="Gender of the user")
    age: int = Field(description="Age of the user")
    dob: str = Field(description="Date of birth of the user in dd/mm/yyyy")

genai_structured_llm = genai_llm.with_structured_output(ResponseFormatter)
genai_structured_llm

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.5, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7dabdfd56480>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'ResponseFormatter', 'description': '', 'parameters': {'properties': {'name': {'description': 'Name of the user', 'type': 'string'}, 'gender': {'description': 'Gender of the user', 'type': 'string'}, 'age': {'description': 'Age of the user', 'type': 'integer'}, 'dob': {'description': 'Date of birth of the user in dd/mm/yyyy', 'type': 'string'}}, 'required': ['name', 'gender', 'age', 'dob'], 'type': 'object'}}}], 'ls_structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'type': 'function', 'function': {'name': 'ResponseFormatter', 'description': '', 'parameters': {'properties': {'nam

In [32]:
messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="My name is atul i was born on 1st jan 2000?"),
]

response = genai_structured_llm.invoke(messages)
response

ResponseFormatter(name='atul', gender='male', age=24, dob='01/01/2000')

In [33]:
response.model_dump()

{'name': 'atul', 'gender': 'male', 'age': 24, 'dob': '01/01/2000'}

In [39]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/dsworkshop74/ds-workshop/refs/heads/main/day-3/Titanic-Dataset%20-%20Titanic-Dataset.csv")
df.head()

,PassengerId,Survived,Pclass,Name,gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [40]:
df.to_csv("titanic.csv",index=False)

In [42]:
from langchain_experimental.agents import create_csv_agent

agent_executor = create_csv_agent(
    genai_llm,
    "titanic.csv",
    verbose=True,
    allow_dangerous_code=True
)

In [43]:
response = agent_executor.run("how many men and woman are there in dataset?")

/tmp/ipython-input-3575607312.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_executor.run("how many men and woman are there in dataset?")




> Entering new AgentExecutor chain...
Thought: The user is asking to count the number of males and females in the dataset. I can achieve this by using the `value_counts()` method on the 'gender' column of the dataframe.
Action: python_repl_ast
Action Input: print(df['gender'].value_counts())gender
male      577
female    314
Name: count, dtype: int64
The output of `df['gender'].value_counts()` clearly shows the counts for 'male' and 'female'. I can now provide the final answer.
Final Answer: There are 577 males and 314 females in the dataset.

> Finished chain.


In [44]:
response

'There are 577 males and 314 females in the dataset.'